<a href="https://colab.research.google.com/github/eikku/JODA-guest-lecture/blob/main/samankaltaisuus_haku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#ks. https://huggingface.co/sentence-transformers
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [14]:
!pip install torch

## Haetaan dataa

In [15]:
#lataa datasetti, lataa omaan käyttöön https://digi.kansalliskirjasto.fi/opendata
url = 'https://raw.githubusercontent.com/eikku/JODA-guest-lecture/main/digi_collectioninfo_202311116.csv' #raw content
df = pd.read_csv(url, sep=";",lineterminator='\n',encoding='utf-8' )

In [16]:
df.head()

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...
1,102,101.0,Käkisalmen ja Laatokan alueen lehdet,images/collections/laatokka_musketti.png,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,De historiska tidningarna och tidskrifterna fr...,Priozersk and Lake Ladoga region newspapers an...,Priozersk and Lake Ladoga region newspapers an...,Tidningar och tidskrifter från Kexholm och Ladoga,https://digi.kansalliskirjasto.fi/collections?...
2,201,501.0,Maantiede ja matkailu,/images/collections/kartat.png,Valikoima Suomea koskevaa tai suomalaisten kir...,Ett urval litteratur om geografi och turism fr...,This is a selection of geography and travel li...,Geography and travel,Geografi och resor,https://digi.kansalliskirjasto.fi/collections?...
3,441,101.0,Bibliotheca Wiburgensis,/images/collections/kaki.jpg,Kokoelma sisältää valikoiman Viipurin Suomalai...,Samlingen innehåller ett urval böcker som publ...,The collection includes a selection of books p...,Bibliotheca Wiburgensis,Bibliotheca Wiburgensis,https://digi.kansalliskirjasto.fi/collections?...
4,501,NaN,Kirjat,images/collections/colbanner6.jpg,NaN,NaN,NaN,Books,Böcker,https://digi.kansalliskirjasto.fi/collections?...


### Otetaan haluamamme sarake ja koodataan se vektoriksi

In [17]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

In [18]:
## Hae malli huggingfacesta
embedder = SentenceTransformer("TurkuNLP/sbert-uncased-finnish-paraphrase")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.93k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/427k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
corpus = df["KUVAUS_FI"].tolist()

In [20]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [21]:
corpus_embeddings

tensor([[-0.0308,  0.2312,  0.9252,  ...,  1.9999,  1.2837,  0.8867],
        [-0.4019,  0.6675,  0.7748,  ...,  1.2126,  1.4767, -0.1489],
        [ 0.5046,  0.3742,  1.5266,  ..., -0.5170, -0.0253,  2.0425],
        ...,
        [ 1.3712,  0.4205,  0.8362,  ...,  0.6010,  1.3584,  0.0940],
        [ 0.8963,  0.8201,  1.0036,  ..., -0.2664,  0.5947,  0.7331],
        [ 1.0879,  0.8370,  1.9932,  ...,  0.2026, -0.1487,  1.7100]])

### Kokeillaan hakua paloissa

In [62]:
query ='Valittuja uudempia aineistoja' # testikuvaus

In [63]:
query_embedding = embedder.encode(query, convert_to_tensor=True) ##embeddataan testikuvaus
hits=util.semantic_search(query_embedding, corpus_embeddings, top_k=3) ## haetaan top 3 samanlaista

In [64]:
hits=hits[0] # saadaan id:t ja samankaltaisuus score
hits

[{'corpus_id': 0, 'score': 0.7574363946914673},
 {'corpus_id': 33, 'score': 0.6856648921966553},
 {'corpus_id': 79, 'score': 0.6523904800415039}]

In [67]:
item = df.iloc[hits[0]["corpus_id"]]

In [68]:
item

ID                                                                    101
PARENT_ID                                                             NaN
NIMI                                              Yhteistyössä toteutetut
BANNER_URL                                  images/collections/sancol.png
KUVAUS_FI               Valittuja uudempia aineistoja digitoidaan ja a...
KUVAUS_SV               Utvalda, ofta nyare material digitaliseras och...
KUVAUS_EN               Selected, usually newer materials are digitise...
NIMI_EN                                  Selections made in collaboration
NIMI_SV                 Verk som digitaliserats med stöd från samarbet...
CollectionPAGELink\r    https://digi.kansalliskirjasto.fi/collections?...
Name: 0, dtype: object

### Paketoidaan funktioihin

In [69]:
def add_embeddings_to_df(df, column_name):
    # Get embeddings for the specified column
    embeddings = embedder.encode(df[column_name].tolist(), show_progress_bar=True)

    # Add embeddings back to the df in a new column 'embeddings'
    df['embeddings'] = list(embeddings)

    return df

In [70]:
add_embeddings_to_df(df,'KUVAUS_FI')

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r,embeddings
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...,"[-0.03082535, 0.23116271, 0.9252266, -0.036827..."
1,102,101.0,Käkisalmen ja Laatokan alueen lehdet,images/collections/laatokka_musketti.png,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,De historiska tidningarna och tidskrifterna fr...,Priozersk and Lake Ladoga region newspapers an...,Priozersk and Lake Ladoga region newspapers an...,Tidningar och tidskrifter från Kexholm och Ladoga,https://digi.kansalliskirjasto.fi/collections?...,"[-0.40191674, 0.66752875, 0.77481115, -0.79700..."
2,201,501.0,Maantiede ja matkailu,/images/collections/kartat.png,Valikoima Suomea koskevaa tai suomalaisten kir...,Ett urval litteratur om geografi och turism fr...,This is a selection of geography and travel li...,Geography and travel,Geografi och resor,https://digi.kansalliskirjasto.fi/collections?...,"[0.5045735, 0.37424886, 1.5266274, -0.2802192,..."
3,441,101.0,Bibliotheca Wiburgensis,/images/collections/kaki.jpg,Kokoelma sisältää valikoiman Viipurin Suomalai...,Samlingen innehåller ett urval böcker som publ...,The collection includes a selection of books p...,Bibliotheca Wiburgensis,Bibliotheca Wiburgensis,https://digi.kansalliskirjasto.fi/collections?...,"[0.211689, 0.711256, 0.8080664, -0.91037655, -..."
4,501,NaN,Kirjat,images/collections/colbanner6.jpg,NaN,NaN,NaN,Books,Böcker,https://digi.kansalliskirjasto.fi/collections?...,"[-0.23689504, 0.48455858, 1.5246029, 0.4104489..."
...,...,...,...,...,...,...,...,...,...,...,...
85,781,101.0,Amerikansuomalaiset lehdet,images/collections/kkgen.png,Amerikansuomalainen kokoelma sisältää Yhdysval...,"Amerikafinländska samlingen ingår finsk-, sven...",The Finnish American Collection includes mater...,Finnish American papers,Amerikafinländska tidningar,https://digi.kansalliskirjasto.fi/collections?...,"[1.7997041, 0.72567254, 2.2175412, -0.11857867..."
86,861,NaN,Vapaat aineistot (lehdet),images/collections/kkgen.png,Vapaasti ladattavat aineistot (OAI-PMH käyttö),Tidningar som fritt kan laddas ner (OAI-PMH),Material for OAI-PMH use.,"Open material (newspapers, journals)",Öppet tillgängliga material (tidningar),https://digi.kansalliskirjasto.fi/collections?...,"[1.3711871, 0.4205001, 0.8362458, 0.36749613, ..."
87,901,NaN,Vapaat aineistot (kirjat),images/collections/kkgen.png,Vapaasti ladattavat aineistot (OAI-PMH käyttö),Böcker som fritt kan laddas ner (OAI-PMH),NaN,Open material (books),Öppet tillgängliga material (böcker),https://digi.kansalliskirjasto.fi/collections?...,"[1.3711871, 0.4205001, 0.8362458, 0.36749613, ..."
88,761,24.0,Käsityölehdet,images/collections/pie5.png,Käsityölehdiksi luokiteltuja lehtiä.,Tidningar med temat hantverk och handarbete.,Journals with genre handicrafts,Handicrafts,Handarbetstidningar,https://digi.kansalliskirjasto.fi/collections?...,"[0.89627635, 0.8201352, 1.003578, -0.394361, -..."


In [71]:
df.head()

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r,embeddings
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...,"[-0.03082535, 0.23116271, 0.9252266, -0.036827..."
1,102,101.0,Käkisalmen ja Laatokan alueen lehdet,images/collections/laatokka_musketti.png,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,De historiska tidningarna och tidskrifterna fr...,Priozersk and Lake Ladoga region newspapers an...,Priozersk and Lake Ladoga region newspapers an...,Tidningar och tidskrifter från Kexholm och Ladoga,https://digi.kansalliskirjasto.fi/collections?...,"[-0.40191674, 0.66752875, 0.77481115, -0.79700..."
2,201,501.0,Maantiede ja matkailu,/images/collections/kartat.png,Valikoima Suomea koskevaa tai suomalaisten kir...,Ett urval litteratur om geografi och turism fr...,This is a selection of geography and travel li...,Geography and travel,Geografi och resor,https://digi.kansalliskirjasto.fi/collections?...,"[0.5045735, 0.37424886, 1.5266274, -0.2802192,..."
3,441,101.0,Bibliotheca Wiburgensis,/images/collections/kaki.jpg,Kokoelma sisältää valikoiman Viipurin Suomalai...,Samlingen innehåller ett urval böcker som publ...,The collection includes a selection of books p...,Bibliotheca Wiburgensis,Bibliotheca Wiburgensis,https://digi.kansalliskirjasto.fi/collections?...,"[0.211689, 0.711256, 0.8080664, -0.91037655, -..."
4,501,NaN,Kirjat,images/collections/colbanner6.jpg,NaN,NaN,NaN,Books,Böcker,https://digi.kansalliskirjasto.fi/collections?...,"[-0.23689504, 0.48455858, 1.5246029, 0.4104489..."


In [95]:
def find_similar_sentences(df, query, top_n):
    # Encode the query to get its embedding as a tensor
    query_embedding = embedder.encode([query], convert_to_tensor=True)

    # Perform semantic search
    hits = util.semantic_search(query_embeddings=query_embedding, corpus_embeddings=corpus_embeddings)[0][:top_n]

    results = []  # Initialize an empty list to store results
    for hit in hits:
        item = df.iloc[hit["corpus_id"]].copy()  # Fetch the data from the DataFrame and make a copy to avoid SettingWithCopyWarning
        item["similarity"] = hit["score"]  # Get the similarity score
        results.append(item.to_dict())  # Append the item as a dictionary (not JSON) to the results list

        # Convert the list of dictionaries into a DataFrame for a tabular format
    results_df = pd.DataFrame(results)

    return results_df


In [96]:
find_similar_sentences(df,query,5)

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r,embeddings,similarity
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...,"[-0.03082535, 0.23116271, 0.9252266, -0.036827...",0.757436
1,462,381.0,Aikakauslehdet (rv),/images/collections/sv_blue.png,Suomen vanhimmat aikakauslehdet,Finlands äldsta tidskrifter.,Finland's earliest journals.,Journals,Tidningar,https://digi.kansalliskirjasto.fi/collections?...,"[-0.35285792, 1.6783265, 1.6069995, -0.5495327...",0.685665
2,741,NaN,Leikekokoelmat,images/collections/kkgen.png,"Tästä kokoelmasta löydät eri teemoista, eri ai...",I denna samling finner du urklippssamlingar so...,"In this collection, you can find clipping coll...",Clipping collections,Urklippssamlingar,https://digi.kansalliskirjasto.fi/collections?...,"[0.23414491, -0.7484715, 2.6163952, 0.3573033,...",0.652390
3,321,61.0,"Historiaa, kirkkohistoriaa ja lähteitä",images/collections/colbanner2.jpg,Digitoidut historiaan ja kirkkohistoriaan liit...,Digitaliserade historiska och kyrkohistoriska ...,Digitized historical and church history relate...,"History, Church History, Sources","Historia, kyrkohistoria, källor",https://digi.kansalliskirjasto.fi/collections?...,"[0.8206587, 0.07376856, 1.3637956, 0.7110134, ...",0.650425
4,266,61.0,Kaunokirjallisuus ja lastenkirjat,images/collections/pilvi.png,Kokoelma sisältää Ruotsin ajan kaunokirjallisu...,Samlingen innehåller skönlitteratur från svens...,The collection includes the part of the fictio...,Fiction and children's literature,Skönlitteratur och barnböcker,https://digi.kansalliskirjasto.fi/collections?...,"[0.7225467, -0.7623577, 2.3292997, -0.51178104...",0.631397
